In [2]:
import os
import gc
import joblib
import pandas as pd
import numpy as np
from sklearn import metrics, preprocessing
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras import utils

In [8]:
def create_model(data,catcols):
    inputs = []
    outputs = []
    for c in catcols:
        num_unique_values = int(data[c].nunique())
        embed_dim = int(min(np.ceil((num_unique_values)/2),50))
        inp = layers.Input(shape=(1,))
        out = layers.Embedding(num_unique_values + 1,embed_dim, name=c)(inp)
        out = layers.SpatialDropout1D(0.3)(out)
        out = layers.Reshape(target_shape=(embed_dim,))(out)
        inputs.append(inp)
        outputs.append(out)
    x = layers.Concatenate()(outputs)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(300,activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(300,activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)
    
    y = layers.Dense(2, activation="softmax")(x)
    
    model = Model(inputs=inputs, outputs=y)
    
    model.compile(loss="binary_crossentropy",optimizer='adam')
    return model

In [10]:
def run(fold):
    df = pd.read_csv("input/cat_train_folds.csv")
    features = [
        f for f in df.columns if f not in ("id","target","kfold")
    ]
    
    for col in features:
        df.loc[:,col] = df[col].astype(str).fillna("NONE")
    
    for feat in features:
        lbl_enc = preprocessing.LabelEncoder()
        df.loc[:,feat] = lbl_enc.fit_transform(df[feat].values)
        
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    
    model = create_model(df,features)
    
    xtrain = [
        df_train[features].values[:,k] for k in range(len(features))
    ]
    
    xvalid = [
        df_valid[features].values[:,k] for k in range(len(features))
    ]
    
    ytrain = df_train.target.values
    yvalid = df_valid.target.values
    
    ytrain_cat = utils.to_categorical(ytrain)
    yvalid_cat = utils.to_categorical(yvalid)
    
    model.fit(xtrain,
             ytrain_cat,
             validation_data=(xvalid,yvalid_cat),
              verbose = 1,
              batch_size = 1024,
              epochs = 3,
             )
    valid_preds = model.predict(xvalid)[:,1]
    print(metrics.roc_auc_score(yvalid,valid_preds))
    K.clear_session()

if __name__ == "__main__":
    run(0)
    run(1)
    run(2)
    run(3)
    run(4)

Train on 480000 samples, validate on 120000 samples
Epoch 1/3
480000/480000 [==============================] - 41s 85us/sample - loss: 0.4815 - val_loss: 0.4396
Epoch 2/3
480000/480000 [==============================] - 26s 54us/sample - loss: 0.4100 - val_loss: 0.3996
Epoch 3/3
480000/480000 [==============================] - 23s 47us/sample - loss: 0.4033 - val_loss: 0.4002
0.7833288005375232
Train on 480000 samples, validate on 120000 samples
Epoch 1/3
480000/480000 [==============================] - 35s 72us/sample - loss: 0.4797 - val_loss: 0.4189
Epoch 2/3
480000/480000 [==============================] - 21s 44us/sample - loss: 0.4102 - val_loss: 0.3968
Epoch 3/3
480000/480000 [==============================] - 21s 44us/sample - loss: 0.4037 - val_loss: 0.3969
0.7873778084490001
Train on 480000 samples, validate on 120000 samples
Epoch 1/3
480000/480000 [==============================] - 27s 56us/sample - loss: 0.4827 - val_loss: 0.4211
Epoch 2/3
480000/480000 [==================